# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 3

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,22,1997,2,1,102,1,0,0,0,0,0,0.88,0.00,0.88,0.88,0.88,1.3,1.3,0.0,1.18,0.00,1.18,1.18,1.18,Matches
2,Che Adams\Che-Adams,eng ENG,FW,Southampton,24,1996,3,3,239,0,1,0,0,1,0,0.00,0.38,0.38,0.00,0.38,0.8,0.8,0.6,0.30,0.23,0.53,0.30,0.53,Matches
3,Semi Ajayi\Semi-Ajayi,ng NGA,DF,West Brom,26,1993,3,3,270,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches
4,Nathan Aké\Nathan-Ake,nl NED,DF,Manchester City,25,1995,2,2,180,1,0,0,0,1,0,0.50,0.00,0.50,0.50,0.50,0.1,0.1,0.0,0.03,0.00,0.03,0.03,0.03,Matches
5,Marc Albrighton\Marc-Albrighton,eng ENG,MF,Leicester City,30,1989,2,0,32,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col='Squad')
teamStats.head()

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Leicester City,1,3,3,0,0,12,4,8,9,6.1,3.3,2.8,0.93,W W W,NaN,Jamie Vardy - 5,Kasper Schmeichel,NaN,2.033333,1.100000,0.332871
Liverpool,2,3,3,0,0,9,4,5,9,8.9,2.8,6.1,2.03,W W W,NaN,Sadio Mané Mohamed Salah - 3,Alisson,NaN,2.966667,0.933333,0.393241
Everton,3,3,3,0,0,8,3,5,9,6.7,1.7,5.0,1.68,W W W,NaN,Dominic Calvert-Lewin - 5,Jordan Pickford,NaN,2.233333,0.566667,0.567414
Aston Villa,4,2,2,0,0,4,0,4,6,2.6,1.6,1.1,0.54,W W,NaN,Ezri Konsa Jack Grealish... - 1,Emiliano Martínez,NaN,1.300000,0.800000,0.449329
Arsenal,5,3,2,0,1,6,4,2,6,4.4,5.1,-0.7,-0.25,W W L,NaN,Alexandre Lacazette - 3,Bernd Leno,NaN,1.466667,1.700000,0.182684


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)
# this is used later
df_online=1

# fetch data locally
#df = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek) + '.csv', index_col=0)
#df_online=0

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,team_name,games played,adjusted points,xG,xG_points,xG_week3,goals_week3,xPoints week 3,clean_sheet_points,xGA_week3,cleansheet_week3,xA,xA_points,xA_week3,assists_week3,adjusted points per game,xPoints week 1,xPoints week 2,next1_xP,next5_xP,next10_xP,value,valuePoints metric,xG_week1,goals_week1,xGA_week1,cleansheet_week1,xA_week1,assists_week1,xG_week2,goals_week2,xGA_week2,cleansheet_week2,xA_week2,assists_week2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,NaN,NaN,0,37605,4.0,NaN,-1,1,-2,2,0.0,549,225,NaN,NaN,0,3,0.5,-0.5,0,Mesut,0.0,0,0,0.0,549,225,False,0.0,548,225,0,NaN,NaN,68,0,0,NaN,0,NaN,37605.jpg,0.0,0,0,Özil,0.8,False,...,a,1,3,0.0,548,224,0,2086,416,33139,3190,0.0,0.0,Özil,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,75.0,75.0,0,39476,NaN,NaN,0,0,-1,1,0.0,444,174,NaN,NaN,0,2,0.4,-0.4,0,Sokratis,0.0,0,0,0.0,458,175,False,0.0,453,175,0,Calf injury - 75% chance of playing,2020-09-11T11:00:08.600094Z,49,0,0,NaN,0,NaN,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,d,1,3,0.0,415,161,0,9750,26,15822,442,0.0,0.0,Sokratis,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,14,100.0,100.0,0,41270,NaN,NaN,0,0,0,0,3.2,233,67,4.0,NaN,0,2,1.2,0.2,1,David,0.7,3,0,2.3,270,93,False,20.2,209,84,91,NaN,2020-09-10T14:30:11.632386Z,55,0,0,NaN,0,NaN,41270.jpg,1.0,0,0,Luiz Moreira Marinho,1.0,False,...,a,1,3,0.0,509,192,2,12317,3172,34453,2564,0.1,0.4,David Luiz,0,Arsenal,2.000000,3.461468,0.0,0.0,0.0,0.0,1.199148,1.461468,3.0,0.0,0.0,0.0,0.0,0.0,1.730734,NaN,1.000000,2.167292,1.699840,1.741253,0.314679,0.737987,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.9,0.0,0.0,0.0
4,1,0,44,NaN,NaN,1,54694,NaN,NaN,0,0,-1,1,52.6,39,27,6.0,NaN,0,3,5.2,4.2,2,Pierre-Emerick,4.7,4,1,19.4,35,19,False,68.8,59,25,270,NaN,NaN,119,0,0,1.0,0,NaN,54694.jpg,4.7,0,0,Aubameyang,29.2,False,...,a,1,3,73.0,29,14,14,230759,33112,1353837,112065,0.4,1.2,Aubameyang,1,Arsenal,2.978723,10.940715,0.5,2.5,0.0,0.0,2.049787,0.544164,3.0,0.0,0.5,1.5,0.0,0.0,3.672954,3.818731,4.149569,3.780792,3.710141,3.761735,0.308652,1.064736,0.4,1.0,0.2,1.0,0.0,0.0,0.1,0.0,1.9,0.0,0.5,1.0
5,0,0,0,75.0,75.0,0,58822,NaN,NaN,-1,1,-2,2,0.0,438,171,NaN,NaN,0,2,0.4,-0.4,0,Cédric,0.0,0,0,0.0,453,172,False,0.0,448,172,0,Calf injury - 75% chance of playing,2020-09-23T09:00:14.881983Z,48,0,0,NaN,0,NaN,58822.jpg,0.0,0,0,Soares,0.3,False,...,d,1,3,0.0,408,158,0,1383,33,23726,2304,0.0,0.0,Cédric,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# assign proper team names for each player
team_names = np.sort(teamStats.index)
# for some reason the fpl team numbers are not alphabetical with leeds and leicester...
if (team_names[8]=='Leeds United') & (team_names[9]=='Leicester City'):
    team_names[8] = 'Leicester City'
    team_names[9] = 'Leeds United'
df['team_name'] = team_names[df['team']-1]

## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [8]:
teamStats['xG per game'] = teamStats['xG'] / teamStats['MP']
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Leicester City,1,3,3,0,0,12,4,8,9,6.1,3.3,2.8,0.93,W W W,NaN,Jamie Vardy - 5,Kasper Schmeichel,NaN,2.033333,1.100000,0.332871
Liverpool,2,3,3,0,0,9,4,5,9,8.9,2.8,6.1,2.03,W W W,NaN,Sadio Mané Mohamed Salah - 3,Alisson,NaN,2.966667,0.933333,0.393241
Everton,3,3,3,0,0,8,3,5,9,6.7,1.7,5.0,1.68,W W W,NaN,Dominic Calvert-Lewin - 5,Jordan Pickford,NaN,2.233333,0.566667,0.567414
Aston Villa,4,2,2,0,0,4,0,4,6,2.6,1.6,1.1,0.54,W W,NaN,Ezri Konsa Jack Grealish... - 1,Emiliano Martínez,NaN,1.300000,0.800000,0.449329
Arsenal,5,3,2,0,1,6,4,2,6,4.4,5.1,-0.7,-0.25,W W L,NaN,Alexandre Lacazette - 3,Bernd Leno,NaN,1.466667,1.700000,0.182684
Crystal Palace,6,3,2,0,1,5,3,2,6,2.7,3.8,-1.1,-0.37,W W L,NaN,Wilfried Zaha - 3,Vicente Guaita,NaN,0.900000,1.266667,0.281769
Leeds United,7,3,2,0,1,8,7,1,6,3.8,6.7,-2.9,-0.95,L W W,NaN,Patrick Bamford - 3,Illan Meslier,NaN,1.266667,2.233333,0.107171
Tottenham,8,3,1,1,1,6,4,2,4,5.8,3.9,1.9,0.63,L W D,NaN,Son Heung-min - 4,Hugo Lloris,NaN,1.933333,1.300000,0.272532
Chelsea,9,3,1,1,1,6,6,0,4,4.5,5.0,-0.5,-0.17,W L D,NaN,Reece James Jorginho... - 1,Kepa Arrizabalaga,NaN,1.500000,1.666667,0.188876


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players' assists based on their xA.

In [9]:
# auxiliary information and variables
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
xA_proportion

0.5344827586206896

In [10]:
df_previous_week = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek-1) + '.csv', index_col=0)
if latest_gameweek == 1:
    df_previous_week['minutes'] = 0
    df_previous_week['xG'] = np.nan
    df_previous_week['xA'] = np.nan
df_previous_week.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,team_name,games played,adjusted points,xG,xG_points,xG_week2,goals_week2,xPoints week 2,clean_sheet_points,xGA_week2,cleansheet_week2,xA,xA_points,xA_week2,assists_week2,adjusted points per game,xPoints week 1,next1_xP,next5_xP,next10_xP,value,valuePoints metric,xG_week1,goals_week1,xGA_week1,cleansheet_week1,xA_week1,assists_week1
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,NaN,NaN,0,37605,4.0,NaN,0,0,-1,1,0.0,541,219,NaN,NaN,0,3,-0.5,0.5,0,Mesut,0.0,0,0,0.0,541,218,False,0.0,540,217,0,NaN,NaN,69,0,0,NaN,0,NaN,37605.jpg,0.0,0,0,Özil,0.9,False,NaN,a,1,3,0.0,538,215,0,1634,639,28323,10500,0.0,0.0,Özil,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,75.0,0.0,0,39476,NaN,NaN,-1,1,-1,1,0.0,414,166,NaN,NaN,0,2,-0.4,0.0,0,Sokratis,0.0,0,0,0.0,429,166,False,0.0,422,166,0,Calf injury - 75% chance of playing,2020-09-11T11:00:08.600094Z,49,0,0,NaN,0,NaN,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,NaN,d,1,3,0.0,378,155,0,9719,9668,15245,11179,0.0,0.0,Sokratis,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,3,100.0,100.0,0,41270,NaN,NaN,0,0,0,0,0.0,493,188,4.0,NaN,0,2,0.0,1.0,1,David,0.5,0,0,0.0,497,188,False,0.0,490,188,1,NaN,2020-09-10T14:30:11.632386Z,55,0,0,NaN,0,NaN,41270.jpg,1.0,0,0,Luiz Moreira Marinho,0.9,False,NaN,a,1,3,0.0,479,186,1,8594,6856,31103,5910,0.1,0.2,David Luiz,0,Arsenal,1.0,2.399751,0.0,0.0,0.0,0.0,1.00000,1.399751,1.9,0.0,0.0,0.0,0.0,0.0,2.399751,NaN,1.325382,1.998728,2.167745,0.436318,1.023257,NaN,NaN,NaN,NaN,NaN,NaN
4,1,0,39,NaN,NaN,1,54694,NaN,NaN,-1,1,-1,1,51.7,19,15,6.0,NaN,0,3,5.5,6.5,5,Pierre-Emerick,6.0,1,1,18.5,13,9,False,64.6,31,15,180,NaN,NaN,119,0,0,1.0,0,NaN,54694.jpg,6.0,0,0,Aubameyang,32.9,False,NaN,a,1,3,69.0,16,6,12,194355,47364,1051163,827602,0.5,1.0,Aubameyang,1,Arsenal,2.0,9.168297,0.5,2.5,0.1,0.0,5.61799,0.699875,1.9,0.0,0.5,1.5,0.5,1.0,4.584148,3.818731,3.437324,4.150217,4.510422,0.385223,1.328878,0.4,1.0,0.2,1.0,0.0,0.0
5,0,0,0,75.0,NaN,0,58822,NaN,NaN,0,0,-1,1,0.0,418,167,NaN,NaN,0,2,-0.4,0.5,0,Cédric,0.0,0,0,0.0,433,167,False,0.0,426,167,0,Calf injury - 75% chance of playing,2020-09-23T09:00:14.881983Z,49,0,0,NaN,0,NaN,58822.jpg,0.0,0,0,Soares,0.3,False,NaN,d,1,3,0.0,383,156,0,1339,438,19821,7066,0.0,0.0,Cédric,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
if latest_gameweek > 1:
    filepath = '../data/fbref/team_stats_week' + str(latest_gameweek-1) + '.csv'
    teamStats_previous_week = pd.read_csv(filepath, index_col='Squad')
    display(teamStats_previous_week.head())
elif latest_gameweek == 1:
    teamStats_previous_week = pd.DataFrame(np.nan, index=team_names, columns=['xGA'])
    display(teamStats_previous_week)

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Leicester City,1,2,2,0,0,7,2,5,6,3.6,2.3,1.2,0.61,W W,NaN,Jamie Vardy - 2,Kasper Schmeichel,NaN,1.80,1.15,0.316637
Everton,2,2,2,0,0,6,2,4,6,5.0,1.3,3.8,1.90,W W,NaN,Dominic Calvert-Lewin - 4,Jordan Pickford,NaN,2.50,0.65,0.522046
Arsenal,3,2,2,0,0,5,1,4,6,3.2,2.1,1.1,0.53,W W,NaN,Alexandre Lacazette - 2,Bernd Leno,NaN,1.60,1.05,0.349938
Liverpool,4,2,2,0,0,6,3,3,6,5.9,1.6,4.3,2.15,W W,NaN,Mohamed Salah - 3,Alisson,NaN,2.95,0.80,0.449329
Crystal Palace,5,2,2,0,0,4,1,3,6,2.2,1.9,0.3,0.17,W W,NaN,Wilfried Zaha - 3,Vicente Guaita,NaN,1.10,0.95,0.386741


In [13]:
def points_for_goal(position):
    if position < 3:
        return 6
    elif position==3:
        return 5
    elif position==4:
        return 4

def points_for_clean_sheet(position):
    if position < 3:
        return 4
    elif position==3:
        return 1
    elif position==4:
        return 0

In [14]:
# column names to be used below
xPoints_column = 'xPoints week ' + str(latest_gameweek)
xG_column = 'xG_week' + str(latest_gameweek) 
goals_column = 'goals_week' + str(latest_gameweek) 
xGA_column = 'xGA_week' + str(latest_gameweek) 
cleansheet_column = 'cleansheet_week' + str(latest_gameweek) 
xA_column = 'xA_week' + str(latest_gameweek) 
assists_column = 'assists_week' + str(latest_gameweek) 

In [15]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    position = df.loc[ix, 'element_type']
    
    # 0.24 is the probability to miss a penalty, which incurs -2 points
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] \
                                        - points_for_goal(position) * (df.loc[ix, 'goals_scored'] - xG) \
                                        - 2 * 0.24 * penalty_attempts    
    df.loc[ix, 'xG'] = xG
    df.loc[ix, 'xG_points'] = points_for_goal(position)*xG - 2 * 0.24 * penalty_attempts
    
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xG']):
                df.loc[ix, xG_column] = df.loc[ix, 'xG'] - df_previous_week.loc[ix, 'xG']
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored'] - df_previous_week.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
            else:
                df.loc[ix, xG_column] = xG
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xG)')

In [16]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[team, 'probability no goals allowed']
    position = df.loc[ix, 'element_type']
    
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] - points_for_clean_sheet(position) * \
                                ( df.loc[ix, 'clean_sheets'] - probability_cleanSheet*df.loc[ix, 'games played'] )
    df.loc[ix, 'clean_sheet_points'] = points_for_clean_sheet(position) * \
                                       df.loc[ix, 'games played'] * probability_cleanSheet
    try:
        if ~np.isnan(teamStats_previous_week.loc[team, 'xGA']):            
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA'] - teamStats_previous_week.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets'] - df_previous_week.loc[ix, 'clean_sheets']
        else:
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets']
        
        if df.loc[ix,'minutes'] - df_previous_week.loc[ix,'minutes'] >= 60:
            probability_cleansheet_thisweek = poisson.pmf(0,df.loc[ix, xGA_column])
            df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - points_for_clean_sheet(position) * \
                                            (df.loc[ix, cleansheet_column] - probability_cleansheet_thisweek)
        
    except KeyError:  
        print(str(ix) + ' is a new index. (xGA)')

In [17]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)
    df.loc[ix, 'xA'] = xA
    df.loc[ix, 'xA_points'] = 3*xA
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xA']):
                df.loc[ix, xA_column] = df.loc[ix, 'xA'] - df_previous_week.loc[ix, 'xA']
                df.loc[ix, assists_column] = df.loc[ix, 'assists'] - df_previous_week.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                    3 * (xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
            else:
                df.loc[ix, xA_column] = xA
                df.loc[ix, assists_column] = df.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                            3*(xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xA)')

In [18]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [21]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','elliott','mattylongstaff']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'mattylongstaff':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.contains('matthew longstaff'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

554 is a new index. (xG)
554 is a new index. (xGA)
554 is a new index. (xA)


In [22]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

if (latest_gameweek > 1) & (df_online==1):
    column_list = []
    for i in range(1,latest_gameweek):
        column_list.append('xPoints week ' + str(i))
        column_list.append('xG_week' + str(i))
        column_list.append('goals_week' + str(i))
        column_list.append('xGA_week' + str(i))
        column_list.append('cleansheet_week' + str(i))
        column_list.append('xA_week' + str(i))
        column_list.append('assists_week' + str(i))
    df = df.join( df_previous_week[column_list])

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,team_name,games played,adjusted points,xG,xG_points,xG_week3,goals_week3,xPoints week 3,clean_sheet_points,xGA_week3,cleansheet_week3,xA,xA_points,xA_week3,assists_week3,adjusted points per game,xPoints week 1,xPoints week 2,next1_xP,next5_xP,next10_xP,value,valuePoints metric,xG_week1,goals_week1,xGA_week1,cleansheet_week1,xA_week1,assists_week1,xG_week2,goals_week2,xGA_week2,cleansheet_week2,xA_week2,assists_week2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,NaN,NaN,0,37605,4.0,NaN,-1,1,-2,2,0.0,549,225,NaN,NaN,0,3,0.5,-0.5,0,Mesut,0.0,0,0,0.0,549,225,False,0.0,548,225,0,NaN,NaN,68,0,0,NaN,0,NaN,37605.jpg,0.0,0,0,Özil,0.8,False,...,a,1,3,0.0,548,224,0,2086,416,33139,3190,0.0,0.0,Özil,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,75.0,75.0,0,39476,NaN,NaN,0,0,-1,1,0.0,444,174,NaN,NaN,0,2,0.4,-0.4,0,Sokratis,0.0,0,0,0.0,458,175,False,0.0,453,175,0,Calf injury - 75% chance of playing,2020-09-11T11:00:08.600094Z,49,0,0,NaN,0,NaN,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,d,1,3,0.0,415,161,0,9750,26,15822,442,0.0,0.0,Sokratis,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,14,100.0,100.0,0,41270,NaN,NaN,0,0,0,0,3.2,233,67,4.0,NaN,0,2,1.2,0.2,1,David,0.7,3,0,2.3,270,93,False,20.2,209,84,91,NaN,2020-09-10T14:30:11.632386Z,55,0,0,NaN,0,NaN,41270.jpg,1.0,0,0,Luiz Moreira Marinho,1.0,False,...,a,1,3,0.0,509,192,2,12317,3172,34453,2564,0.1,0.4,David Luiz,0,Arsenal,2.000000,3.461468,0.0,0.0,0.0,0.0,1.199148,1.461468,3.0,0.0,0.0,0.0,0.0,0.0,1.730734,NaN,1.00000,2.167292,1.699840,1.741253,0.314679,0.737987,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.9,0.0,0.0,0.0
4,1,0,44,NaN,NaN,1,54694,NaN,NaN,0,0,-1,1,52.6,39,27,6.0,NaN,0,3,5.2,4.2,2,Pierre-Emerick,4.7,4,1,19.4,35,19,False,68.8,59,25,270,NaN,NaN,119,0,0,1.0,0,NaN,54694.jpg,4.7,0,0,Aubameyang,29.2,False,...,a,1,3,73.0,29,14,14,230759,33112,1353837,112065,0.4,1.2,Aubameyang,1,Arsenal,2.978723,10.940715,0.5,2.5,0.0,0.0,2.049787,0.544164,3.0,0.0,0.5,1.5,0.0,0.0,3.672954,3.818731,5.61799,3.780792,3.710141,3.761735,0.308652,1.064736,0.4,1.0,0.2,1.0,0.0,0.0,0.1,0.0,1.9,0.0,0.5,1.0
5,0,0,0,75.0,75.0,0,58822,NaN,NaN,-1,1,-2,2,0.0,438,171,NaN,NaN,0,2,0.4,-0.4,0,Cédric,0.0,0,0,0.0,453,172,False,0.0,448,172,0,Calf injury - 75% chance of playing,2020-09-23T09:00:14.881983Z,48,0,0,NaN,0,NaN,58822.jpg,0.0,0,0,Soares,0.3,False,...,d,1,3,0.0,408,158,0,1383,33,23726,2304,0.0,0.0,Cédric,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
388,Kane,3.010753,28,9.3,30.382759,10.091416
251,Mané,3.012048,25,8.3,26.684460,8.859241
272,De Bruyne,2.000000,15,7.5,17.147150,8.573575
254,Salah,3.010753,28,9.3,25.620502,8.509667
166,Richarlison,3.013699,22,7.3,24.909655,8.265476
164,Calvert-Lewin,3.009709,31,10.3,24.500000,8.140323
224,Vardy,2.990654,32,10.7,23.180000,7.750813
68,Maupay,2.988506,26,8.7,21.836552,7.306846
12,Martínez,2.000000,19,9.5,14.594632,7.297316


In [24]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [25]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total xG: ' + str(total_xG))
print('Total goals: ' + str(total_goals))
print('Total xG / total goals: ' + str(total_xG/total_goals))

Total xG: 80.7
Total goals: 100
Total xG / total goals: 0.807
